In [ ]:
# import cProfile
# import collections
import datetime

import os
# import sys
import time

# import numpy as np
import pandas as pd

import pickle

# from datetime import datetime
# from dotenv import load_dotenv

import gc

pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})
format = '%d/%m/%Y %H:%M'

# 3. Concatenate all playlists

## 3.1 Raw playlists

In [ ]:
# Concatenate all playlists
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_full.pkl'
txt = 'All playlists concatenated - ' + data_path
if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        playlists_full_by_user = pickle.load(f)
else:
    print('Saved: ' + txt)
    playlists_full_by_user = []
    for i in range(0,100,1):
        data_path_aux = os.getcwd() + '\\data\\external\\raw_2_playlists_' + str(i) + '.pkl'
        if os.path.exists(data_path_aux):
            print(data_path_aux)
            with open(data_path_aux, 'rb') as f:
                playlist = pickle.load(f)
            playlists_full_by_user.extend(playlist)

    with open(data_path, 'wb') as f:
        pickle.dump(playlists_full_by_user, f)

print(len(playlists_full_by_user))

## 3.2 Playlist dictionary

In [ ]:
def get_artists(raw_artist):
    artists = {}
    for a in raw_artist:
        artists.update({a['id']: {'id': a['id'],
                'name': a['name']}})
    return(artists);

# a = get_artists(playlists_full_by_user[6]['tracks']['items'][2]['track']['album']['artists'])
# a

In [ ]:
def get_track_details(raw_track):
    if raw_track['track'] is not None:
        artists = get_artists(raw_track['track']['album']['artists'])
        track = raw_track['track']
        data = {track['id']:
                    {
                    'id': track['id'],
                    'name': track['name'],
                    'added_at': raw_track['added_at'],
                    'popularity': track['popularity'],
                    'artists': artists,

                    'album': {
                        'id': track['album']['id'],
                        'name': track['album']['name'],
                        'release_date': track['album']['release_date']
                    }
                    }
                }
        return(data);

a = get_track_details(playlists_full_by_user[6]['tracks']['items'][82])
a
# a = get_track_details(playlists_full_by_user[6]['tracks']['items'][83])
# playlists_full_by_user[6]['tracks']['items'][83]

In [ ]:
def get_playlist_detail(playlist):
    tracks = []
    for i, item in enumerate(playlist['tracks']['items']):
        if not(item['track'] is None or item['track']['id'] is None):
            track = get_track_details(item)
            tracks.append(track)

    data = {playlist['id']:
                {
                'user_id': playlist['owner']['id'],
                'playlist_id': playlist['id'],
                'total_tracks': len(tracks),
                'tracks': tracks
                }
        }

    return(data);
# t = get_playlist_detail(playlists_full_by_user[4])
# t

In [ ]:
# Concatenate all playlists
data_path = os.getcwd() + '\\data\\external\\raw_3_playlists_dict.pkl'
txt = 'All playlists as dictionary - ' + data_path
if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        playlists_dict = pickle.load(f)
else:
    # Concatenate all playlists as dictionary
    n = len(playlists_full_by_user)

    playlists_dict = {}
    for i, playlist in enumerate(playlists_full_by_user):
        if i % 100 == 0:
            print("%s - %d/%d - LEN: %d" % (datetime.datetime.now().strftime(format), i, n, len(playlists_dict)))

        play_dict = get_playlist_detail(playlist)
        playlists_dict.update({playlist['id']: play_dict})

    #     playlists_dict['7AU3xd5nGS3JT5ZVNcXBsM']
    len(playlists_dict)  # CHECK WHY THE COUNT OF PLAYLISTS DECREASED
    
    print('Saved: ' + txt)
    with open(data_path, 'wb') as f:
        pickle.dump(playlists_dict, f)

print(len(playlists_dict))  # CHECK WHY THE COUNT OF PLAYLISTS DECREASED

In [ ]:
# def get_artists(raw_artist):
#     artists_id = []
#     artists = []
#     for a in raw_artist:
#         artists_id.append(a['id'])
#         artists.append(a['name'])
        
#     return(artists_id, artists);

# a = get_artists(playlists_full_by_user[6]['tracks']['items'][2]['track']['album']['artists'])
# a





# 12149404267
# 4jwLkSVrnathscvWtxy1Fa
# # b['2E124GmJRnBJuXbTb4cPUB']



# for j in range(1,2100,1):
#     for k in playlists_full_by_user[j]['tracks']['items'][k]['track']['album']['artists']:
#         print(k)
#         print(j)
#         print(k)
#         a = get_artists(playlists_full_by_user[j]['tracks']['items'][k]['track']['album']['artists'])
# a





# for i, item in enumerate(playlists_full_by_user[8]['tracks']['items']):
# #     if item['id'] == '2E124GmJRnBJuXbTb4cPUB':
# #         print(i)
    
#         for a in item['track']:
#             print(a)
#             b = a
            
# #         print(item['id'])
# #         print(playlists_full_by_user[i]['id'])

## 3.3 Tracks dictionary

In [ ]:
# All tracks
data_path = os.getcwd() + '\\data\\external\\raw_4_tracks_dict.pkl'
txt = 'All tracks as a dictionary - ' + data_path

if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        tracks_dict = pickle.load(f)
else:
    
    # Extract the user and playlist IDs
    tracks_dict = {}
    for i, pid in enumerate(playlists_dict):
        pl = playlists_dict.get(pid)
        for j, tr in enumerate(pl['tracks']):
            tid = tr['id']
    #         print(str(pid)  + ':' + str(i) + ' - ' + str(tid) + ':' +str(j))
            if tid not in tracks_dict.keys():
                tr['in_playlist'] = [pid]
                tracks_dict.update({tid: tr})
            else:
                curr_track = tracks_dict.get(tid)
                curr_in_playlist = curr_track['in_playlist'].copy()
                curr_in_playlist.append(pid)
                tr['in_playlist'] = curr_in_playlist
                tracks_dict.update({tid: tr})
    #             .append(tid)
    #             tr['in_playlist'].append(pid)

    print('Saved: ' + txt)
    with open(data_path, 'wb') as f:
        pickle.dump(tracks_dict, f)

print(len(tracks_dict))  # CHECK WHY THE COUNT OF PLAYLISTS DECREASED

In [ ]:
a = playlists_dict.get('2iuTbpx2K4mMq3WNdkzj8Q')
a

In [ ]:
a['tracks'][0]
# b = a['playlist_id'].get('5KawlOMHjWeUjQtnuRs22c')
# b
# print(len(a['tracks']))

## 3.4 Artists dictionary

In [ ]:
gc.collect()

In [ ]:
# len(playlists_dict)
# for item in playlists_dict:
#     print(item)

In [ ]:
# a = playlists_dict.get('2iuTbpx2K4mMq3WNdkzj8Q')
# a = playlists_dict.get('4jwLkSVrnathscvWtxy1Fa')
# a

# print(len(a))
# b = a['tracks']
# 12149404267
# 4jwLkSVrnathscvWtxy1Fa
# b['2E124GmJRnBJuXbTb4cPUB']

# c = b['artists_id'][0]
# c

In [ ]:
# All artists
data_path = os.getcwd() + '\\data\\external\\raw_5_artists_dict.pkl'
txt = 'All artists as a dictionary - '

if os.path.exists(data_path):
    print(txt + 'Loaded:' + data_path)
    with open(data_path, 'rb') as f:
        artists_dict = pickle.load(f)
else:
    # Extract the user and playlist IDs
    artists_dict = {}
    for i, pid in enumerate(playlists_dict):
        pl = playlists_dict.get(pid)
        for j, tr in enumerate(pl['tracks']):
#             print(j)
#             print(tr)
#             print(tr['artists'])
#             print()
#             for art in tr['artists']:
                if i == 1:
                    print(k)
                    print(art)
#                 aid = a['id'][k]
#                 art = tr['name'][k]
      #         print(str(pid)  + ':' + str(i) + ' - ' + str(tid) + ':' +str(j))
                if art['id'] not in artists_dict.keys():
                    art['in_playlist']: [pid]
                    artists_dict.update({aid: art})
                else:
                    curr_artist = artists_dict.get(aid)
                    curr_in_playlist = curr_artist['in_playlist'].copy()
#                     curr_in_playlist.append(pid)
#                     tr['in_playlist'] = curr_in_playlist
#                     artists_dict.update({tid: tr})
    #             .append(tid)
    #             tr['in_playlist'].append(pid)
    
#         print(txt + 'Saved:' + data_path)
#         with open(data_path, 'wb') as f:
#             pickle.dump(tracks_dict, f)

len(tracks_dict)

## 3.4 Albums dictionary

In [ ]:
# a = playlists_dict.get('2iuTbpx2K4mMq3WNdkzj8Q')
# # b = a.get('5KawlOMHjWeUjQtnuRs22c')
# print(len(a['tracks']))
# # a

In [ ]:
# Extract the user and playlist IDs
albums_dict = {}
for i, pid in enumerate(playlists_dict):
    pl = playlists_dict.get(pid)
    for j, tr in enumerate(pl['albums']):
        tid = tr['album_id']
        curr_in_playlist = []
#         print(str(pid)  + ':' + str(i) + ' - ' + str(tid) + ':' +str(j))
        if tid not in albums_dict.keys():
            tr['in_playlist'] = [pid]
            albums_dict.update({tid: tr})
        else:
            curr_album = albums_dict.get(tid)
            curr_in_playlist = curr_album['in_playlist'].copy()
            curr_in_playlist.append(pid)
            tr['in_playlist'] = curr_in_playlist
            albums_dict.update({tid: tr})
#             .append(tid)
#             tr['in_playlist'].append(pid)

len(albums_dict)

# Ranking

In [ ]:
# tracks_rank = df_track.groupby(['tracks_id','tracks']).agg(
#     popularity=('popularity','max'),
#     tracks_count=('tracks_id','count')
#     )

# min_max = MinMaxScaler()
# tracks_rank['count_scaled'] = min_max.fit_transform(tracks_rank[["tracks_count"]])

# tracks_rank["rank"] = tracks_rank['tracks_count'].rank(method='average',ascending=False)

# tracks_rank = tracks_rank.sort_values('tracks_count',ascending=False)

# tracks_rank

In [ ]:
# album_rank = df_album.groupby(['albums_id','albums']).agg(album_count=('albums_id','count'))

# min_max = MinMaxScaler()
# album_rank['count_scaled'] = min_max.fit_transform(album_rank[["album_count"]])

# album_rank["rank"] = album_rank['album_count'].rank(method='average',ascending=False)

# album_rank = album_rank.sort_values('album_count',ascending=False)

# album_rank

In [ ]:
# df_artist.head
# artist_rank = df_artist.groupby(['artists_id','artists']).agg(artist_count=('artists_id','count'))

# min_max = MinMaxScaler()
# artist_rank['count_scaled'] = min_max.fit_transform(artist_rank[["artist_count"]])

# artist_rank["rank"] = artist_rank['artist_count'].rank(method='average',ascending=False)

# artist_rank = artist_rank.sort_values('artist_count',ascending=False)

# artist_rank

In [ ]:
# tracks_cartesian = []
# for element in itertools.product(s_tracks_id,s_tracks_id):
#     tracks_cartesian.append(element)

# https://stackoverflow.com/questions/11144513/cartesian-product-of-x-and-y-array-points-into-single-array-of-2d-points
# def cartesian_product(*arrays):
#     la = len(arrays)
#     dtype = np.result_type(*arrays)
#     arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
#     for i, a in enumerate(numpy.ix_(*arrays)):
#         arr[...,i] = a
#     return arr.reshape(-1, la)


# # tracks_cartesian = cartesian_product(*(s_tracks_id * 2))

In [ ]:
# for key, item in df_test.items():
#     print(key)
#     if key == 'tracks_id':
#         print(item)

In [ ]:
# # Define count_entries()
# def count_entries(csv_file,c_size,colname):
#     """Return a dictionary with counts of
#     occurrences as value for each key."""
    
#     # Initialize an empty dictionary: counts_dict
#     counts_dict = {}

#     # Iterate over the file chunk by chunk
#     for chunk in pd.read_csv(csv_file,chunksize=c_size):

#         # Iterate over the column in DataFrame
#         for entry in chunk[colname]:
#             if entry in counts_dict.keys():
#                 counts_dict[entry] += 1
#             else:
#                 counts_dict[entry] = 1

#     # Return counts_dict
#     return counts_dict

# # Call count_entries(): result_counts
# result_counts = count_entries('tweets.csv',10,'lang')

# # Print result_counts
# print(result_counts)


In [ ]:
# a = playlists_full_by_user[0]
# pid = a['id']
# owner = a['owner']
# tracks = a['tracks']

# track = tracks['items'][0]['track']['album']
# track
# # type(track[0])
# # artist =track[0]
# for i, feat in enumerate(track):
#     print(feat)
# #     print(type(feat))
# #     print()

# m = playlists_full_by_user[83]
# # m
# # n = m['tracks']['items']
# # n
# p = n[2]['track']
# p['']['name']

# # a = playlists_full_by_user[6]['tracks']['items']
# # a[0]['track']['album']['artists']
# # for i in range(83,200,1):
# #     print (i)
# #     print(a[i]['track']['album']['artists'])


# # for a in m:
# #     print(a)
# # m['items']
# # p = n[2]['track']['album']['artists']
# # print(p)